___


```c++
#include <iostream>
#include <cstdint>

int global = 0;

int main()
{
    int* heap = (int*) malloc(sizeof(int));

    std::cout << std::hex << (uint64_t) main << '\n';
    std::cout << std::hex << (uint64_t) &global << '\n';
    std::cout << std::hex << (uint64_t) heap << '\n';
    std::cout << std::hex << (uint64_t) &heap << '\n';

    // hack, how to not close terminal of your program
    char c;
    std::cin >> c;
    return 0;
}
```

In [3]:
!g++ -O0 mem.cpp -o mem --std=c++11 && ./mem

55d676648229
55d67664b27c
55d677775eb0
7ffd3c4e07e0
^C


In [ ]:
!ps ax | grep mem


##### /proc/.../maps

```
ps ax | grep mem
```

```
00400000-00401000 r-xp 00000000 08:01 2362492
        /home/mt/work/tmp/mem
00601000-00602000 r--p 00001000 08:01 2362492
        /home/mt/work/tmp/mem
00602000-00603000 rw-p 00002000 08:01 2362492
        /home/mt/work/tmp/mem
0189c000-018ce000 rw-p 00000000 00:00 0
        [heap]
7f66aaa53000-7f66aabc5000 r-xp 00000000 08:01 6826866
        /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21
7f66aadc5000-7f66aadcf000 r--p 00172000 08:01 6826866
        /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21
7f66aadcf000-7f66aadd1000 rw-p 0017c000 08:01 6826866
        /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.21
7ffd55900000-7ffd55921000 rw-p 00000000 00:00 0
        [stack]
7ffd55952000-7ffd55954000 r--p 00000000 00:00 0
        [vvar]
7ffd55954000-7ffd55956000 r-xp 00000000 00:00 0
        [vdso]
ffffffffff600000-ffffffffff601000 r-xp 00000000 00:00 0
        [vsyscall]
```

#### Виртуальная память

![](images/virtual_memory.png)

- Память делится на страницы
- Страница может находится в оперативной памяти или на внешнем носителе
- Трансляция из физического адреса в виртуальный и обратно выполняется через специальные таблицы: PGD (Page Global Directory), PMD (Page Middle Directory) и PTE (Page Table Entry). В PTE хранятся физические адреса страниц
- Для ускорения трансляции адресов процессор хранит в кеше таблицу TLB (Translation lookaside buffer)
- Если обращение к памяти не может быть оттранслировано через TLB, процессор обращается к таблицам страниц и пытается загрузить PTE оттуда в TLB. Если загрузка не удалась, процессор вызывает прерывание Page Fault
- Обработчик прерывания Page Fault находится в подсистеме виртуальной памяти ядра ОС и может загрузить требуемую страницу с внешнего носителя в оперативную память


```
1 такт = 1 / частота процессора
1 / 3 GHz = 0.3 ns
                                             0.3 ns
L1 cache reference                           0.5 ns
Branch mispredict                            5   ns
```
> Неудачный if ()
```
L2 cache reference                           7   ns
Mutex lock/unlock                           25   ns
Main memory reference                      100   ns
```
> Кроме задержки (latency) есть понятие пропускной способности (throughput, bandwidth). В случае чтения из RAM - 10-50 Gb/sec
```
Compress 1K bytes with Zippy             3,000   ns
Send 1K bytes over 1 Gbps network       10,000   ns
Read 4K randomly from SSD              150,000   ns
Read 1 MB sequentially from memory     250,000   ns
Round trip within same datacenter      500,000   ns
Read 1 MB sequentially from SSD      1,000,000   ns
HDD seek                            10,000,000   ns
Read 1 MB sequentially from HDD     20,000,000   ns
Send packet CA->Netherlands->CA    150,000,000   ns
```
Источник: [https://gist.github.com/jboner/2841832](https://gist.github.com/jboner/2841832)

Иллюстрация: [https://github.com/Kobzol/hardware-effects](https://github.com/Kobzol/hardware-effects)

### Выводы из таблицы
1. Стараться укладывать данные в кеш
2. Минимизировать скачки по памяти
3. В условиях основной веткой делать ветку которая выполняется с большей вероятностью
___


```C++
Вопрос с предыдущей пары:
sizeof(array);

int i[][2] = { { 1, 2 }, { 3, 4 }, { 5, 6 } };
sizeof(i); // == 24 ?

int i[][2] = { { 1, 2 }, { 3, 4 }, { 5, 6 }, { 7, 8 } };
sizeof(i); // == 32 ?

int i[][3] = { { 1, 2 }, { 3, 4 }, { 5, 6 }};
sizeof(i); // == 36 ?

int i[][3] = { { 1, 2 }, { 3, 4 }, { 5, 6 }};
sizeof(i[1]); // == 12 ?

```

```C++
int i[][2] = { { 1, 2 }, { 3, 4 }, { 5, 6 } };
int* j = (int*) i;
j[5] // ?? 
j[6] // ?? (ниже вывод)


using matrix = int(*)[2];
matrix k = (matrix) i;
k[5] // ??
```

g++ -c main.cpp
g++ main.o -o main
./main
j[5]=6
j[6]=249830912
k[0]=0x7ffc86634f10
k[0][1]=2

### !!! Рассмотреть код решённых задач 

___

### Классы управления памятью и областью видимости в C++

Характеризуются тремя понятиями:
1. **Время жизни**
> Продолжительность хранения данных в памяти
2. **Область видимости**
> Части кода из которых можно получить доступ к данным
3. **Связывание (linkage)**
> Если к данным можно обратиться из другой единицы трансляции — связывание внешнее (external), иначе связывание внутреннее (internal)


#### Автоматический/регистровый (register)

| Время жизни | Область видимости | Связывание |
| --- | --- | --- |
| Автоматическое (блок) | Блок | Отсутствует |

```c++
{
	int i = 5;
}

if (true)
{
	register int j = 3;
}

for (int k = 0; k < 7; ++k)
{
}
```

#### Статический без связывания

| Время жизни | Область видимости | Связывание |
| --- | --- | --- |
| Статическое | Блок | Отсутствует |

```c++
void foo()
{
	// где лежит переменная?
	static int j = 3;
}
```

```c++
// например
int counter()
{
	static int j = 3;
	j++;
	return j;
}
```

> Инициализируется при первом обращении


#### Статический с внутренним связыванием

| Время жизни | Область видимости | Связывание |
| --- | --- | --- |
| Статическое | Файл | Внутреннее |

> math.cpp
```c++
static float pi_value = 3.1415;

float getPi()
{
    return pi_value;
}
```
> math.h
```C++
float getPi();
```

> main.cpp
```c++
#include <iostream>
#include "math.h"
int main()
{
    std::cout << getPi(); // no access to `pi_value`
    return 0;
}
```

> Инициализируется до входа в main

#### Статический с внешним связыванием

| Время жизни | Область видимости | Связывание |
| --- | --- | --- |
| Статическое | Файл | Внешнее |

```c++
// *.cpp
int i = 0;
```
```c++
// *.h
extern int i;
```
> Инициализируется в cpp файле. Опасно, т.к. видно во всей программе


### Зантяие 3. Функции, пространства имён

### Пространства имен

Проблема:

```c++
// math.h
double cos(double x);
```

```c++
// ваш код
double cos(double x);
```

##### Решение в стиле С:

```c++
// ваш код
double fastCos(double x);
```

##### Решение:

```c++
namespace fast
{
    double cos(double x);
}

fast::cos(x);
cos(x); // вызов из math.h
```

#### Поиск имен

- Проверка в текущем namespace
- Если имени нет и текущий namespace глобальный - ошибка
- Рекурсивно поиск в родительском namespace


```c++
void foo() {} // ::foo - :: глобальный неймспейс

namespace A
{
    void foo() {} // A::foo

    namespace B
    {
        void bar() // A::B::foo
        {
            foo(); // == A::foo
            ::foo(); // foo()
        }
    }
}
```


#### Ключевое слово using

Добавляет имена из указанного namespace в текущий namespace.

```c++
void foo()
{
    using namespace A;
    // видимо все из A
}
```

```c++
void foo()
{
    using namespace A::foo;
    // видима только A::foo()
}
```

```c++
void foo()
{
    namespace ab = A::B;
    ab::bar(); // A::B::bar()
}
```


##### using может приводить к проблемам

```c++
using namespace fast;

cos(x); // ???
cos(x); // ???
```

> Не используйте using namespace в заголовочных файлах!

**Пример 1:**

**Замечание:** `using namespace X` - заполнить текущее пространство имён до закрывающей скобки именами из пространства X.
Вызывающая программа:

```c++
#include "json_parser.h"

int main()
{
    std::string s = "123";

    // не скомпилируется, т.к. в глобальном пространстве имён нет функции read_int
    std::cout << read_int(s);
    
    // скомпилируется
    std::cout << json_parser::input_processing::read_int(s);
    
    // скомпилируется, т.к. глобальное пространство имён расширено
    using namespace json_parser::input_processing;
    std::cout << read_int(s);
    
    return 0;    
}
```

**Пример 2:**

```c++
int main() {
    std::cout << 1 << std::string("23");
    return 0;
}
```

или

```c++
using namespace std; 
// имена из std доступны в глобальном пространстве имён до конца файла

int main() {
    cout << 1 << string("23");
    return 0;
}
```

или

```c++
int main()
{
    using namespace std; 
    // имена из std доступны в глобальном пространстве имён до конца main

    cout << 1 << string("23");
    return 0;
}
```

___


### Ссылки

Ссылка - псевдоним объекта.

Главное отличие от указателя - ссылка должна быть проинициализирована при объявлении и до конца своей жизни ссылается только на один объект.

```c++
int a = 1;
int b = 2;
int* ptr = nullptr;
ptr = &a;
ptr = &b;
int& ref; // Ошибка
int& ref = a; // ref ссылается на a
ref = 5; // Теперь a == 5
ref = b; // ref не стала указывать на b,
         // мы просто скопировали значение из b в a
ref = 7; // a == 7, b == 2
```
```c++
int a = 2;
int* ptr = nullptr;
int*& ptrRef = ptr; // ptrRef ссылается на ptr
ptrRef = &a; // теперь ptr хранит адрес a
```



### Передача аргументов в функции

#####  По значению

```c++
void foo(int x)
{
    x = 3;
}

int x = 1;
foo(x);
// x == 1

void bar(BigObject o) { ... }
```

- В функции окажется копия объекта, ее изменение не отразится на оригинальном объекте
- Копировать большие объекты может оказаться накладно



##### По ссылке

```c++
void foo(int& x)
{
    x = 3;
}

int x = 1;
foo(x);
// x == 3

void bar(BigObject& o) { ... }
```

- Копирования не происходит, все изменения объекта внутри функции отражаются на объекте
- Следует использовать, если надо изменить объект внутри функции

```c++
void swap(int& x, int& y)
{
    int tmp = x;
    x = y;
    y = tmp;
}
```


##### По константной ссылке

```c++
void foo(const int& x)
{
    x = 3; // ошибка компиляции
}

void bar(const BigObject& o) { ... }
```
- Копирования не происходит, при попытке изменения объекта будет ошибка
- Большие объекты выгодней передавать по ссылке, маленькие - наоборот



```c++
int countObject(time_t* fromDate, time_t* toDate)
{
    const auto begin =
        fromDate == nullptr
            ? objects_.begin()
            : objects_.findFirst(fromDate);
}
```


##### По универсальной ссылке

```c++
void foo(int&& x) { ... }
void bar(BigObject&& o) { ... }
```
Поговорим позже


##### Перегрузка функций

```c++
void print(int x) // 1
{
    std::cout << x << std::endl;
}

void print(bool x) // 2
{
    std::cout << (x ? "true" : "false") << std::endl;
}

print(10); // 1
print(true); // 2
```


##### Опасность перегрузки

```c++
void print(const std::string& x) // 3
{
    std::cout << "string" << std::endl;
}

print("hello!"); // 2 const char* приводится к bool, потому что приводимость ищется сначала среди базовых типов
```

> Перегруженная функция - декорированная функция

```c++
std::string convert_to_string(int x);       // 1
std::string convert_to_string(unsigned x);  // 2
std::string convert_to_string(float x);     // 3

std::cout << convert_to_string(5);    // 1
std::cout << convert_to_string(5u);   // 2
std::cout << convert_to_string(5.f);  // 3
```

Для такого набора функций `convert_to_string` компилятор (clang 10.0) сгенерирует символы:
* `_Z17convert_to_stringB5cxx11i`
* `_Z17convert_to_stringB5cxx11j`
* `_Z17convert_to_stringB5cxx11f`

Т.е. тип аргумента - часть имени функции при компиляции.


#### Указатель на функцию

```c++
void foo(int x)
{
}

typedef void (*FooPtr)(int);

FooPtr ptr = foo;
ptr(5);
```

```c++
using FooPtr = void(*)(int);
```


### Лямбда-функции

```c++
auto lessThen3 = [](int x) { return x < 3; };

if (lessThen3(x)) { ... }

auto it_element = std::find(vec.begin(), vector.end(), lessThen3)
```
##### Синтаксис

```c++
[список_захвата](список_параметров) { тело_функции }
```
```c++
[список_захвата](список_параметров) -> тип_возвращаемого_значения
{ тело_функции }
```

___

##### Передача аргументов в функцию

Передача по значению - создание копии аргумента

```c++
float min_value(std::vector<float> x);
```

Передача по ссылке - работа с аргументом

```c++
float min_value(std::vector<float>& x);
```

Передача по const-ссылке - работа с аргументом и запрет на модификацию

```c++
float min_value(const std::vector<float>& x);
```

**Упражнение:** что здесь происходит с аргументами?

```c++
float sqr(const float x);

float min_value(std::vector<float>* x);
float min_value(const std::vector<float>* x);
float min_value(std::vector<float>* const x);
```

**Вопрос:** Когда лучше передавать по значению, а когда - по ссылке?

<br />

##### Возвращаемое значение

Функция ничего не возвращает:

```c++
void say_hello(const std::string& name)
{
    std::cout << "hello, " << name;
}
```

Возврат результата через возвращаемое значение (предпочтительный вариант):

```c++
std::vector<std::string> make_team()
{
    return { "Bifur", "Bofur", "Bombur", "Oin",
             "Gloin", "Fili", "Nori", "Dori",
             "Dwalin", "Ori", "Balin", "Kili" };
}
```

Возврат результата через аргумент (менее предпочтительный вариант в силу меньшей читаемости):

```c++
bool append_teamlead(Point3D location, std::vector<std::string>& team)
{
    if (is_inside(location, get_village("Shire")))
    {
        team.push_back("Thorin");
        return true;
    }
    return false;
}
```

Сравните:
    
```c++
// легко читается, что есть результат функции
std::vector<std::string> team = make_team();

// не очевидно, что результатом функции является
// изменение второго аргумента, нужно лезть в
// документацию или реализацию, чтобы понять
// замысел автора
append_teamlead(get_current_location(), team);

```

<br />

##### Ошибки при работе с аргументами и возвращаемыми значениями

```c++
std::string* find_dwarf(const std::vector<std::string>& team, const std::string& name)
{
    for (const std::string& dwarf : team)
        if (dwarf == name)
            return &dwarf;
    return nullptr;
}

// usage 1
std::vector<std::string> team = make_team();
if (std::string* ptr = find_dwarf(team, "Kuzya"))
    std::cout << *ptr;

// usage 2
if (std::string* ptr = find_dwarf(make_team(), "Balin"))
    std::cout << *ptr;  // ???

// usage 3
if (std::string* ptr = find_dwarf({"Ori", "Gloin", "Balin"}, "Balin"))
    std::cout << *ptr;  // ???
```

__Вопрос__: что будет напечатано программой ниже?

Показать пример на godbolt.org на clang 8.0.0 с разными оптимизациями, чтобы наглядно продемонстрировать ub

```c++
#include <iostream>
#include <string>

const std::string& f(const bool x,
                     const std::string& s1,
                     const std::string& s2)
{
    return x ? s1 : s2;
}

int main()
{
    const std::string& s = f(true, "123", "12345");
    std::cout << s << endl;
    return 0;
}
```

<br />

##### Значения аргументов по умолчанию

Можно задавать значения аргументов по умолчанию:

```c++
std::string convert_to_string(int value, int base = 10);
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n");
```

использование:

```c++
std::string s1 = convert_to_string(42);    // 42
std::string s2 = convert_to_string(42, 2); // 101010

std::string song = join_strings({"In the town where I was born",
                                 "Lived a man who sailed to sea",
                                 "And he told us of his life",
                                 "In the land of submarines",
                                });
                                 
std::string sentence = join_strings({"Nobody",
                                     "expects",
                                     "the",
                                     "spanish",
                                     "inquisition",
                                    },
                                    " ");
```

Но такие аргументы должны быть последними:

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines);  // compilation ERROR!
```

```c++
std::string join_strings(const std::vector<std::string>& strings,
                         const std::string& sep = "\n",
                         bool skip_empty_lines = false);  // OK
```